# Sentiment


In [2]:
import json
import requests
import pandas as pd
import torch
from tqdm import tqdm
from dotenv import dotenv_values
from pymongo import MongoClient
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

## 1) Italian BERT Sentiment model
https://huggingface.co/neuraly/bert-base-italian-cased-sentiment 

In [3]:
# estraggo il DataFrame dal file excel
df = pd.read_excel('../POI con recensioni/POI.xlsx')

In [4]:
df

,id,POI,rating,recensione
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume..."
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...
...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina..."
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...


In [5]:
# carico il tokenizzatore
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")

In [6]:
bert_model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
bert_model.eval()  # Imposta il modello in modalità valutazione

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

1. Estrae la recensione dalla colonna recensione.
2. Codifica la recensione utilizzando un tokenizer, parte di un modello di linguaggio preaddestrato.
3. Trasforma la sequenza codificata in un tensore e lo prepara per l'input al modello.
4. Passa il tensore al modello per ottenere i logit, che sono le uscite del modello prima di applicare la funzione di attivazione.
5. Applica una funzione di softmax sui logit per ottenere le probabilità associate a ciascuna classe (probabilità negative, neutre e positive).
6. Memorizza le probabilità negative, neutre e positive ottenute nella tabella DataFrame df.
7. Determina la classe di sentimento prevista utilizzando la probabilità massima tra negative, neutre e positive.
8. Memorizza la classe di sentimento prevista nella tabella DataFrame df.

In [7]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Itera attraverso ogni riga del DataFrame 'df', mostrando una barra di progresso con tqdm
    sentence = row.recensione

    # Tokenizza la frase della recensione usando il tokenizer specificato
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

    # Crea un tensore, usa .cuda() per trasferire il tensore alla GPU se disponibile
    tensor = torch.tensor(input_ids).long().unsqueeze(0)
    if torch.cuda.is_available():
        tensor = tensor.cuda()
        bert_model.cuda()

    # Chiama il modello BERT e ottiene i logits (uscite grezze prima dell'applicazione della funzione softmax)
    with torch.no_grad():
        logits = bert_model(tensor)
    
    # Rimuove la dimensione fittizia del batch dai logits
    logits = logits.logits.squeeze(0)

    # Applica la funzione softmax per ottenere le probabilità
    proba = nn.functional.softmax(logits, dim=0)

    # Estrae le probabilità per le classi negativo, neutro e positivo
    negative, neutral, positive = proba

    # Assegna le probabilità alle colonne 'negative', 'neutral' e 'positive' nel DataFrame
    df.loc[index, 'negative'] = proba[0].item()
    df.loc[index, 'neutral'] = proba[1].item()
    df.loc[index, 'positive'] = proba[2].item()


  0%|          | 0/5335 [00:00<?, ?it/s]

100%|██████████| 5335/5335 [04:37<00:00, 19.20it/s]


In [8]:
proba

tensor([0.2985, 0.0351, 0.6664], device='cuda:0')

Il "tokenizer" è una componente fondamentale quando si lavora con modelli di linguaggio naturale (NLP). La sua funzione principale è quella di convertire il testo in una forma che il modello possa comprendere. Questo processo coinvolge solitamente la suddivisione del testo in parole o sotto-unità semantiche (come i token) e la successiva conversione di questi token in numeri, che possono essere utilizzati come input per il modello. Questo processo di tokenizzazione può anche includere altre operazioni, come la rimozione della punteggiatura, la gestione delle maiuscole e minuscole, e altro ancora.

Il "tensore", d'altra parte, è una struttura dati fondamentale nelle librerie di calcolo numerico, come TensorFlow o PyTorch, utilizzate per l'implementazione di modelli di machine learning e deep learning. In termini semplici, un tensore può essere considerato come un array multidimensionale, che può essere un vettore (1D), una matrice (2D) o una struttura multidimensionale di ordine superiore. I tensori sono essenziali poiché consentono di rappresentare i dati e le operazioni matematiche eseguite sui dati in modo efficiente e parallelo, specialmente quando si lavora con grandi quantità di dati.

In questo contesto il "tokenizer" viene utilizzato per convertire il testo delle recensioni in sequenze di token numerici, mentre il "tensore" viene utilizzato per rappresentare queste sequenze di token in una forma che il modello di machine learning possa elaborare.

In [9]:
df.head()

,id,POI,rating,recensione,negative,neutral,positive
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497


In [25]:
# Determino l'etichetta del sentiment sulla base della probabilità più alta
df['sentiment_pred'] = df[['negative', 'positive', 'neutral']].idxmax(axis=1)


In [26]:
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred,BERT
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,positive,positive
...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000215,0.001434,0.998351,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000135,0.001645,0.998220,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000398,0.003119,0.996483,positive,positive


In [28]:
df.to_excel('../sentiment/BERT.xlsx', index=False)

In [13]:
df = pd.read_excel('../POI con recensioni/POI.xlsx')
df.rating.value_counts()

rating
5    2637
4    1551
3     710
1     249
2     188
Name: count, dtype: int64

In [29]:
df = pd.read_excel('../sentiment/BERT.xlsx')
df.sentiment_pred.value_counts()

AttributeError: 'DataFrame' object has no attribute 'BERT'

In [18]:
df.to_csv("../sentiment/BERT_value_counts.csv", index=False, sep='\t')

### Calcolo dell'Accuracy

In [31]:
df = pd.read_excel('../sentiment/BERT.xlsx')

In [32]:
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,positive
...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000215,0.001434,0.998351,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000135,0.001645,0.998220,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000398,0.003119,0.996483,positive


In [35]:
from sklearn.metrics import classification_report

df['GS']=0
df.loc[df['rating'] >= 3, 'GS']  = 'positive'
df.loc[df['rating'] <= 2, 'GS']  = 'negative'
df

,id,POI,rating,recensione,negative,neutral,positive,sentiment_pred,GS
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,positive,positive
...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000215,0.001434,0.998351,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000135,0.001645,0.998220,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000398,0.003119,0.996483,positive,positive


In [36]:
print(classification_report(df['GS'], df['sentiment_pred']))

              precision    recall  f1-score   support

    negative       0.62      0.72      0.67       437
     neutral       0.00      0.00      0.00         0
    positive       0.98      0.85      0.91      4898

    accuracy                           0.84      5335
   macro avg       0.53      0.52      0.53      5335
weighted avg       0.95      0.84      0.89      5335



c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [37]:
# Calcola il classification report
report = classification_report(df['GS'], df['sentiment_pred'], output_dict=True)

# Creare un DataFrame pandas dal classification report
df_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/BERT_classification_report.xlsx'
df_report.to_excel(excel_filename, index=True)

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

## 2) FEEL - IT
https://huggingface.co/MilaNLProc/feel-it-italian-sentiment?text=Mi+piaci.+Ti+amo

Il modello Feel-it-Italian-Sentiment esegue l'analisi del sentiment sull'italiano. Viene utilizzato il modello UmBERTo sul nuovo set di dati (cioè FEEL-IT) ottenendo prestazioni all'avanguardia su diversi corpus di riferimento.

I dati sono stati raccolti annotando tweet di un’ampia gamma di argomenti. In totale, ci sono 2037 tweet annotati con un'etichetta emozionale.